In [ ]:
#LSTM Code from backup

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from itertools import product
import pickle  # For saving best parameters

warnings.filterwarnings('ignore')

# Set the seed for reproducibility
SEED = 42

# Python random module
import random
random.seed(SEED)

# NumPy random generator
np.random.seed(SEED)

# PyTorch random generators
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
#torch.cuda.manual_seed_all(SEED)  # If using multiple GPUs

# Ensure deterministic behavior in cuDNN (may slow down training)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load the dataset
file_path = r"D:\Reproducing the values\MR_at_test\Datasets\MR_30\test_a\505_c1_test_30.csv"
data = pd.read_csv(file_path)

# Convert the 'datetime' feature to datetime object with the correct format
data['datetime'] = pd.to_datetime(data['datetime'], format='%d/%m/%Y %H:%M')

# Handle missing values in 'value' column by replacing empty strings with NaN
data['value'] = pd.to_numeric(data['value'], errors='coerce')

# Encode categorical features
label_encoders = {}
for column in ['weekday', 'hour', 'is_weekend', 'is_working_hour', 'season']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the 'value' column
scaler = StandardScaler()
data['value_normalized'] = scaler.fit_transform(data[['value']])

# Separate rows with missing values
missing_data = data[data['value'].isna()]
data_train = data.dropna()

# Split the dataset into features and labels
features = data_train.drop(columns=['value', 'value_normalized', 'datetime', 'meter_no'])
labels = data_train['value_normalized']

# Split the dataset into training, validation, and test sets
train_flag = True
prefix = '505_LSTM_Seoul_30_'
#-----------------------------------
if train_flag == True:
    # First split: 80% train+val, 20% test
    X_temp, X_test, y_temp, y_test = train_test_split(features, labels, test_size=0.2, random_state=SEED)
    # Second split: 87.5% of temp (70% of total) train, 12.5% of temp (10% of total) val
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=SEED)  # 0.125 of 0.8 = 0.1 of total

    train_data = pd.DataFrame(X_train)
    train_data['label'] = y_train
    val_data = pd.DataFrame(X_val)
    val_data['label'] = y_val
    test_data = pd.DataFrame(X_test)
    test_data['label'] = y_test

    train_data.to_csv(prefix + "X_y_train.csv", index=False)
    val_data.to_csv(prefix + "X_y_val.csv", index=False)
    test_data.to_csv(prefix + "X_y_test.csv", index=False)
else:
    loaded_train_data = pd.read_csv(prefix + "X_y_train.csv")
    loaded_val_data = pd.read_csv(prefix + "X_y_val.csv")
    loaded_test_data = pd.read_csv(prefix + "X_y_test.csv")

    X_train = loaded_train_data.drop('label', axis=1)
    X_val = loaded_val_data.drop('label', axis=1)
    X_test = loaded_test_data.drop('label', axis=1)
    y_train = loaded_train_data['label']
    y_val = loaded_val_data['label']
    y_test = loaded_test_data['label']
#-----------------------------------

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_val = torch.tensor(X_val.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x.unsqueeze(1), (h0, c0))  # Reshape input to (batch_size, sequence_length, input_size)
        out = self.fc(out[:, -1, :])
        return out

# Training Loop for LSTM Model with Early Stopping
def train(model, train_loader, val_loader, optimizer, epochs, patience):
    model.train()
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.MSELoss()(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = nn.MSELoss()(outputs, labels)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

    return train_losses, val_losses

# Prepare the dataset for training and validation
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)  # Optional, for clarity
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)


# Grid search for hyperparameter tuning for LSTM
param_grid = {
    'hidden_size': [32,64,128],
    'num_layers': [2,3,4],
    'learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64, 128]
}

best_val_loss = float('inf')
best_params = {}
save_dir = r"D:\Reproducing the values\MR_at_test\Models"
os.makedirs(save_dir, exist_ok=True)
if train_flag == True:
    for hidden_size, num_layers, lr, batch_size in product(param_grid['hidden_size'], param_grid['num_layers'], param_grid['learning_rate'], param_grid['batch_size']):
        print(f"Training with hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={lr}, batch_size={batch_size}")
        model = LSTMModel(X_train.shape[1], hidden_size, num_layers)
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Prepare the dataset for training and validation
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        train_losses, val_losses = train(model, train_loader, val_loader, optimizer, epochs=150, patience=10)

        if min(val_losses) < best_val_loss:
            best_val_loss = min(val_losses)
            best_params = {
                'hidden_size': hidden_size,
                'num_layers': num_layers,
                'learning_rate': lr,
                'batch_size': batch_size
            }

    # Ensure the directory exists

    # Save the best parameters
    pkl_save_path = os.path.join(save_dir,prefix + "best_params.pkl")
    with open(pkl_save_path, "wb") as f: #write binary (file is open for writing in binary mode)
        pickle.dump(best_params, f) #serialize an object and write to a file
    print(f"Best parameters saved to: {pkl_save_path}")
    print(f"Best parameters saved to: {best_params}")

    # Train the best model with the best parameters
    best_hidden_size = best_params['hidden_size']
    best_num_layers = best_params['num_layers']
    best_learning_rate = best_params['learning_rate']
    best_batch_size = best_params['batch_size']

model = LSTMModel(X_train.shape[1], best_hidden_size, best_num_layers)
optimizer = optim.Adam(model.parameters(), lr=best_learning_rate)

# Preparing the dataset for training and validation with the best batch size
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)


# Save the trained model
model_save_path = os.path.join(save_dir, prefix + "trained_.pth")




if(train_flag == True):
    train_losses, val_losses = train(model, train_loader, val_loader, optimizer, epochs=250, patience=20)
    torch.save(model.state_dict(), model_save_path)
    print(f"Trained model saved to: {model_save_path}")
else:
    state_dict = torch.load(model_save_path, map_location=torch.device('cpu'))

    adjusted_state_dict = {key.replace("model.", ""): value for key, value in state_dict.items()}
    try:
        model.load_state_dict(adjusted_state_dict)
    except RuntimeError as e:
        print(f"Error in loading state_dict: {e}")
        print("Please verify that the model architecture matches the saved state dictionary.")
        raise



sns.set()
plt.figure(figsize=(6, 4))
plt.plot(train_losses, label="Training loss")
plt.plot(val_losses, label="Validation loss")
plt.legend(loc="upper right")
plt.xlabel("Training steps")
plt.title("Training and Validation Loss")
plt.savefig(r"D:\Reproducing the values\MR_at_test\Datasets\MR_30\505_lstm_30\505_LSTM_Seoul_30a_Plot.png")
plt.show()

# Ensure the directory exists
save_dir = r"D:\Reproducing the values\MR_at_test\Models"
os.makedirs(save_dir, exist_ok=True)




# Impute missing values in the dataset
missing_features = missing_data.drop(columns=['value', 'value_normalized', 'datetime', 'meter_no'])
missing_features = torch.tensor(missing_features.values, dtype=torch.float32)

model.eval()
with torch.no_grad():
    imputed_values = model(missing_features).numpy()

# Scale back the imputed values to the original scale
imputed_values = scaler.inverse_transform(imputed_values)

# Create a new column '502_LSTM_Seoul_40' and fill it with the original 'value' column
data['505_LSTM_Seoul_30'] = data['value']

# Update the '502_LSTM_Seoul_40' column with the imputed values at the missing indices
data.loc[missing_data.index, '505_LSTM_Seoul_30'] = imputed_values[:, 0]

# Save the updated dataset to a CSV file
output_path = r"D:\Reproducing the values\MR_at_test\Datasets\MR_30\505_lstm_30\505_LSTM_30.csv"
data.to_csv(output_path, index=False)
print(f"Updated dataset saved to: {output_path}")

# Compute the RMSE score on the test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test).numpy()

# Scale back the test predictions to the original scale
y_test_inverse = scaler.inverse_transform(y_test.cpu())
test_predictions_inverse = scaler.inverse_transform(test_predictions)

# Plot for after training
sns.set()
plt.figure(figsize=(6, 4))
plt.plot(y_test_inverse, label="Target output")
plt.plot(test_predictions_inverse[:, 0], label="LSTM output", linestyle="dashed")
plt.legend(loc="upper right")
plt.title("After Training")
plt.savefig(r"D:\Reproducing the values\MR_at_test\Datasets\MR_30\505_lstm_30\505_LSTM_30a.png")
plt.show()

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_test_inverse, test_predictions_inverse[:, 0]))
print(f'RMSE: {rmse}')

# Calculate the correlation between 'ytrue' and imputed 'value'
correlation = data['ytrue'].corr(data['505_LSTM_Seoul_30'])
print(f"Correlation: {correlation}")

# Calculate the correlation between true test values and predicted test values
test_correlation = pd.Series(y_test_inverse.flatten()).corr(pd.Series(test_predictions_inverse[:, 0]))
print(f"Test Correlation: {test_correlation}")


Training with hidden_size=32, num_layers=2, learning_rate=0.001, batch_size=32
Epoch 1, Train Loss: 0.17509911536501768, Val Loss: 0.0055814395684187544
Epoch 2, Train Loss: 0.003639632030950466, Val Loss: 0.009510369582280312
Epoch 3, Train Loss: 0.0017016883549731431, Val Loss: 0.002208469990538816
Epoch 4, Train Loss: 0.001171417818845597, Val Loss: 0.001057293792319012
Epoch 5, Train Loss: 0.001114648040755458, Val Loss: 0.0006780120153825296
Epoch 6, Train Loss: 0.0010447209572701493, Val Loss: 0.00035508083975363574
Epoch 7, Train Loss: 0.0007345198383954348, Val Loss: 0.0009721864564248604
Epoch 8, Train Loss: 0.0007643388674007334, Val Loss: 0.0007486345536675151
Epoch 9, Train Loss: 0.0006072335493912405, Val Loss: 0.0020191956659115543
Epoch 10, Train Loss: 0.0007204423448795552, Val Loss: 0.00015035529457684926
Epoch 11, Train Loss: 0.0007404963286503776, Val Loss: 0.0002628369164160351
Epoch 12, Train Loss: 0.0005287572168429952, Val Loss: 0.000277295805536185
Epoch 13, Tra